In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-18 07:20:13.762902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 07:20:13.780262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8463] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 07:20:13.785645: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argume

In [2]:
import os
import pandas as pd
from pathlib import Path
import json
import cv2
from moviepy.editor import VideoFileClip
import tempfile
from tqdm import tqdm

def prepare_messages(process_path):
    return [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/home/ubuntu/video-6.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {
                "type": "text", 
                "text": """Is there an indication of suspicious store activity in this video such as shoplifting or not. 
                    Reply with two fields
                    answer: Yes or No
                    reason: Your reason"""
            },
        ],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text", 
                "text": """Yes\nReason: The video shows a man picking office supplies from a table and hiding them in his pockets in a suspicious manner"""
            },
        ],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/home/ubuntu/video-3.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {
                "type": "text", 
                "text": """Is there an indication of suspicious store activity in this video such as shoplifting or not. 
                    Reply with two fields
                    answer: Yes or No
                    reason: Your reason"""
            },
        ],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text", 
                "text": """No\nReason: The video shows one man casually shopping with a basket and another browsing items on the shelf, no strong signs of shoplifting"""
            },
        ],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": process_path,
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {
                "type": "text", 
                "text": """
                    Is there a strong indication of suspicious store activity in this video such as shoplifting or not. 
                    Reply with two fields
                    answer: Yes or No
                    reason: Your reason
                    """
            },
        ],
    }
]
    

def get_video_duration(video_path):
    """Get the duration of a video in seconds."""
    try:
        clip = VideoFileClip(video_path)
        duration = clip.duration
        clip.close()
        return duration
    except Exception as e:
        raise Exception(f"Error getting video duration: {str(e)}")

def trim_video(input_path, output_path, duration=30):
    """Trim video to specified duration in seconds."""
    try:
        with VideoFileClip(input_path) as clip:
            trimmed_clip = clip.subclip(0, duration)
            trimmed_clip.write_videofile(output_path, 
                                       codec='libx264', 
                                       audio=False,
                                       logger=None)  
    except Exception as e:
        raise Exception(f"Error trimming video: {str(e)}")

def sample_frames_dynamic_v1(video_path, output_path, target_frames=30):
    try:
        cap = cv2.VideoCapture(video_path)
        fps = 1 #cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = total_frames / fps
        print(f"Video duration: {duration:.2f} seconds, Total frames: {total_frames}")

        sampling_interval = max(1, int(total_frames / target_frames))
        print(f"Sampling every {sampling_interval} frames to extract {target_frames} frames.")
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % sampling_interval == 0 and saved_count < target_frames:
                out.write(frame)
                saved_count += 1
            frame_count += 1
            if saved_count >= target_frames:
                break
        cap.release()
        out.release()
    except Exception as e:
        raise Exception(f"Error sampling and recreating video: {str(e)}")

def sample_frames_dynamic_v2(video_path, output_path, target_frames=30, target_duration=30, output_fps=30):
    try:
        cap = cv2.VideoCapture(video_path)
        input_fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        input_duration = total_frames / input_fps      
        target_frames = target_duration * output_fps
        sampling_interval = max(1, total_frames / target_frames)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, output_fps, (width, height))
        frame_count = 0
        saved_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count >= saved_count * sampling_interval and saved_count < target_frames:
                out.write(frame)
                saved_count += 1
            frame_count += 1
            if saved_count >= target_frames:
                break
        
        cap.release()
        out.release()
        
    except Exception as e:
        raise Exception(f"Error sampling and recreating video: {str(e)}")

def process_video_directory(video_dir, output_csv, max_duration=30):
    results = []
    temp_dir = tempfile.mkdtemp()
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    for video_file in tqdm(video_files):
        try:
            video_path = os.path.join(video_dir, video_file)
            print(f"\nProcessing {video_file}...")
            
            duration = get_video_duration(video_path)
            print(f"Video duration: {duration:.2f} seconds")
            
            if duration > max_duration:
                sampled_frames_dir = os.path.join(temp_dir, f"resampled_{(video_file)}")
                sample_frames_dynamic_v1(video_path, sampled_frames_dir)
                process_path = sampled_frames_dir 
                print("Frame sampling complete")
            else:
                process_path = video_path

            ## uncomment for 30-second trimming

            # if duration > max_duration:
            #     print(f"Video longer than {max_duration} seconds, trimming...")
            #     temp_video_path = os.path.join(temp_dir, f"trimmed_{video_file}")
            #     trim_video(video_path, temp_video_path, max_duration)
            #     process_path = temp_video_path
            # else:
            #     process_path = video_path
            
            messages = prepare_messages(process_path)

            text = processor.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            inputs = inputs.to("cuda")

            # Generate response
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]

            response_lines = output_text.strip().split('\n')
            
            answer = response_lines[0].split(': ')[-1].strip()
            reason = response_lines[1].split(': ')[1].strip()

            results.append({
                'filename': video_file,
                'duration': duration,
                'trimmed': duration > max_duration,
                'anomaly': answer,
                'reason': reason,
            })
            
            print(f"Analysis complete - Anomaly detected: {answer}")
            
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            results.append({
                'filename': video_file,
                'duration': -1,
                'trimmed': False,
                'anomaly': 'ERROR',
                'reason': str(e)
            })

    try:
        import shutil
        shutil.rmtree(temp_dir)
        print(f"\nCleaned up temporary directory: {temp_dir}")
    except Exception as e:
        print(f"Error cleaning up temporary directory: {str(e)}")

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\nResults saved to {output_csv}")
    return df


VIDEO_DIR = "/home/ubuntu/content/drive/MyDrive/shoplifting-videos"  
OUTPUT_CSV = "video_analysis_results.csv"
MAX_DURATION = 30 

results_df = process_video_directory(VIDEO_DIR, OUTPUT_CSV, MAX_DURATION)
print("\nAnalysis Summary:")
print(f"Total videos processed: {len(results_df)}")
print(f"Videos trimmed: {len(results_df[results_df['trimmed']])}")
print(f"Anomalies detected: {len(results_df[results_df['anomaly'] == 'Yes'])}")
print(f"Processing errors: {len(results_df[results_df['anomaly'] == 'ERROR'])}")

  0%|          | 0/49 [00:00<?, ?it/s]


Processing Shoplifting052_x264.mp4...
Video duration: 254.88 seconds
Video duration: 7646.00 seconds, Total frames: 7646
Sampling every 254 frames to extract 30 frames.
Frame sampling complete


qwen-vl-utils using torchvision to read video.
  2%|▏         | 1/49 [00:09<07:44,  9.67s/it]

Analysis complete - Anomaly detected: Yes

Processing Shoplifting034_x264.mp4...
Video duration: 397.90 seconds
Video duration: 11937.00 seconds, Total frames: 11937
Sampling every 397 frames to extract 30 frames.
Frame sampling complete


  4%|▍         | 2/49 [00:19<07:35,  9.69s/it]

...

Analysis complete - Anomaly detected: Yes

Processing Shoplifting003_x264.mp4...
Video duration: 360.62 seconds
Video duration: 10817.00 seconds, Total frames: 10817
Sampling every 360 frames to extract 30 frames.
Frame sampling complete


 98%|█████████▊| 48/49 [06:45<00:08,  8.19s/it]

Analysis complete - Anomaly detected: Yes

Processing video-4.mp4...
Video duration: 19.98 seconds


100%|██████████| 49/49 [06:51<00:00,  8.40s/it]

Analysis complete - Anomaly detected: Yes

Cleaned up temporary directory: /tmp/tmpw7xafohx

Results saved to video_analysis_results.csv

Analysis Summary:
Total videos processed: 49
Videos trimmed: 40
Anomalies detected: 48
Processing errors: 0
